# Cross validation using classical grid search

In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Activation
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


Using TensorFlow backend.


In [6]:
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    width=32
    height=32
    depth=3
    classes=10
    model = Sequential()
    inputShape = (height, width, depth)
    
    model.add(Conv2D(32,(3,3),input_shape=inputShape,activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    
    model.add(Dense(1024, activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(512, activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    
    print("[INFO] compiling model...")
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])
    
    return model


In [7]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
print(trainX.shape)
print(testX.shape)

[INFO] loading CIFAR-10 data...
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [8]:
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

#This reshaping of the train and test labels were required to use KerasClassifier wrapper in scikitlearn

Ytrain = (np.reshape(trainY, (np.product(trainY.shape)))).astype('float64')
Ytest = (np.reshape(testY, (np.product(testY.shape)))).astype('float64')

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY1 = lb.fit_transform(trainY)
testY1 = lb.transform(testY)

# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer","dog", "frog", "horse", "ship", "truck"]

In [ ]:
model=KerasClassifier(build_fn=create_model,epochs=10,batch_size=64,verbose=1)
#grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
inits = ['glorot_uniform','normal','uniform']

epochs = [5,10]
batches = [32,64]

param_grid = dict(optimizer=optimizers, epochs=epochs,batch_size=batches, init=inits)


grid = GridSearchCV(estimator=model,param_grid=param_grid)

grid_result = grid.fit(trainX,Ytrain)

[INFO] compiling model...
Epoch 1/5
33333/33333 [==============================] - 187s - loss: 1.8222 - acc: 0.3297   
Epoch 2/5
33333/33333 [==============================] - 189s - loss: 1.3996 - acc: 0.4976   
Epoch 3/5
33333/33333 [==============================] - 190s - loss: 1.2312 - acc: 0.5685   
Epoch 4/5
33333/33333 [==============================] - 186s - loss: 1.1534 - acc: 0.6007   
Epoch 5/5
33312/33333 [============================>.] - ETA: 0s[INFO] compiling model...
Epoch 1/5
33333/33333 [==============================] - 185s - loss: 1.8349 - acc: 0.3149   
Epoch 2/5
33333/33333 [==============================] - 185s - loss: 1.4122 - acc: 0.4914   
Epoch 3/5
33333/33333 [==============================] - 186s - loss: 1.2294 - acc: 0.5672   
Epoch 4/5
33333/33333 [==============================] - 187s - loss: 1.1339 - acc: 0.6068   
Epoch 5/5
33312/33333 [============================>.] - ETA: 0s[INFO] compiling model...
Epoch 1/5
33334/33334 [===================

33334/33334 [==============================] - 183s - loss: 1.4007 - acc: 0.5016   
Epoch 3/5
33334/33334 [==============================] - 182s - loss: 1.2166 - acc: 0.5755   
Epoch 4/5
33334/33334 [==============================] - 182s - loss: 1.1296 - acc: 0.6100   
Epoch 5/5
33334/33334 [==============================] - 46s    
[INFO] compiling model...
Epoch 1/5
33333/33333 [==============================] - 188s - loss: 1.8541 - acc: 0.3025   
Epoch 2/5
33333/33333 [==============================] - 187s - loss: 1.3783 - acc: 0.4971   
Epoch 3/5
33333/33333 [==============================] - 187s - loss: 1.1986 - acc: 0.5730   
Epoch 4/5
33333/33333 [==============================] - 187s - loss: 1.0761 - acc: 0.6161   
Epoch 5/5
33312/33333 [============================>.] - ETA: 0s[INFO] compiling model...
Epoch 1/5
33333/33333 [==============================] - 189s - loss: 1.7263 - acc: 0.3541   
Epoch 2/5
33333/33333 [==============================] - 188s - loss: 1.3243 

33334/33334 [==============================] - 187s - loss: 0.9344 - acc: 0.6732   
Epoch 8/10
33334/33334 [==============================] - 187s - loss: 0.8890 - acc: 0.6872   
Epoch 9/10
33334/33334 [==============================] - 187s - loss: 0.8446 - acc: 0.7033   
Epoch 10/10
33334/33334 [==============================] - 45s    
[INFO] compiling model...
Epoch 1/10
33333/33333 [==============================] - 182s - loss: 1.8176 - acc: 0.3259   
Epoch 2/10
33333/33333 [==============================] - 182s - loss: 1.3720 - acc: 0.5075   
Epoch 3/10
33333/33333 [==============================] - 182s - loss: 1.2029 - acc: 0.5785   
Epoch 4/10
33333/33333 [==============================] - 181s - loss: 1.1314 - acc: 0.6104   
Epoch 5/10
33333/33333 [==============================] - 181s - loss: 1.0908 - acc: 0.6264   
Epoch 6/10
33333/33333 [==============================] - 182s - loss: 1.1050 - acc: 0.6275   
Epoch 7/10
33333/33333 [==============================] - 182s 

33333/33333 [==============================] - 182s - loss: 1.2290 - acc: 0.5678   
Epoch 4/10
33333/33333 [==============================] - 182s - loss: 1.1505 - acc: 0.6035   
Epoch 5/10
33333/33333 [==============================] - 182s - loss: 1.1254 - acc: 0.6136   
Epoch 6/10
33333/33333 [==============================] - 181s - loss: 1.1187 - acc: 0.6216   
Epoch 7/10
33333/33333 [==============================] - 181s - loss: 1.1087 - acc: 0.6304   
Epoch 8/10
33333/33333 [==============================] - 181s - loss: 1.1384 - acc: 0.6233   
Epoch 9/10
33333/33333 [==============================] - 181s - loss: 1.1230 - acc: 0.6296   
Epoch 10/10
33333/33333 [==============================] - 45s    
[INFO] compiling model...
Epoch 1/10
33334/33334 [==============================] - 182s - loss: 1.8231 - acc: 0.3297   
Epoch 2/10
33334/33334 [==============================] - 181s - loss: 1.4015 - acc: 0.4956   
Epoch 3/10
33334/33334 [==============================] - 181s 

In [ ]:
#Summarize the results

print("Best %f using %s" %(grid_result.best_score_,grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']

params = grid_result.cv_results_['params']

for mean,stdev,param in zip(means,stds,params):
    print("%f (%f) with %r" %(mean,stdev,param))
